In [ ]:
%run Utilities.ipynb

In [ ]:
a_lines = {
    'TR': [
        """- Arkadaşım Bora'ya göndereceğim mesajın
  güvenli olması için tek kullanımlık
  bir anahtar göndermek istiyorum.
  Bunun için rastgele pozisyonda
  fotonlar kullanacağım.""",
        """- Elimdeki rastgele bit'ler:""",
        """- Elimdeki rastgele yönler:""",
        """- Bora'ya gönderilen kuantum durum:""",
        """- Bunun sonunda ikimizin de elinde,
  {eldeki} bit'lik bir dizi oldu.
  (Hedefimiz ise {hedef} bit'lik bir diziydi.)""",
        """- Uyuşmazlık belli bir oranın üzerindeyse,
  bunu kullanmak yerine baştan başlarız."""        
    ],
    'EN': [
        """- I want to send a one-time pad
  to my friend Bob to ensure security.
  For this, I will use photons in
  random polarizations.""",
        """- My random classical bits:""",
        """- My random polarizations:""",
        """- Quantum state sent to Bob:""",
        """In the end, we both have
  the same {actual}-bit string.
  (Our target was {target}-bits.)""",
        """If discrepancy is above a certain
  level, we discard this and start over."""
    ]
}

b_lines = {
    'TR': [
        """- Ayşe'nin mesajıyla aynı boyda
  rastgele yön belirliyorum:""",
        """- Bu rastgele yöne göre ölçümlerim:""",
        """- Eğer kusur veya dinlenme riski varsa,
  bunun belli bir kısmı karşılaştırılabilir."""
    ],
    'EN': [
        """- I generate random directions,
  same length as Alice's message.""",
        """- Measurements according to random directions:""",
        """- If there's risk of flaw or eavesdropping,
  some of these bits can be compared."""
    ]
}

c_lines = {
    'TR': [
        """Rastgele yönlerimizi klasik kanalda karşılaştırıyoruz:""",
        """Aynı yönlerde bit'ler de aynı olacaktır:"""
    ],
    'EN': [
        """We compare our directions over a classical channel:""",
        """When directions match, bits will match as well:"""
    ]
}

names = {
    'TR': ['Ayşe', 'Bora'],
    'EN': ['Alice', 'Bob']
}

supported_languages = {'TR', 'EN'}

def BB84_DEMO(target_bits:int, lang:str='EN', speed:int=1):
    """
    lang:   Language desired for the demo. If not supported
        or blank, defaults to English. Uses ISO 639-1 coding.
    speed:  Speed desired for the text change,
        with 1 slowest and 5 fastest (no waiting between lines.)
    """
    target = 2*target_bits
    lang = lang.upper()
    if(lang not in supported_languages):
        lang = 'EN'
    if(speed<1): speed=1
    if(speed>5): speed=5
    speed = (5-speed)/2
    
    alice = iter(a_lines[lang])
    bob = iter(b_lines[lang])
    common = iter(c_lines[lang])

    h_length = 44
    # Each actor has approximately h_length wide columns.
    names = names[lang]
    print(names[0].upper().center(h_length)+names[1].upper().center(h_length))
    print("="*(2*h_length))
    
    print(next(alice))
    print()
    
    alice_bits = grb(target)
    alice_directions = grb(target)
    sleep(2)
    
    print(next(alice))
    i = 0
    while(i*32<len(alice_bits)):
        print('    '+alice_bits[32*i:min(32*(i+1),len(alice_bits))])
        i += 1
    print()
    
    print(next(alice))
    aqs = polarizer(alice_directions)
    i = 0
    while(i*32<len(aqs)):
        print('    '+aqs[32*i:min(32*(i+1),len(aqs))])
        i += 1
    print()
    
    bob_directions = grb(target)
    bob_bits = ''
    
    sleep(2)
    q_comm = '|'
    # Ayşe'nin rastgele bit ve yönleri var,
    # Bora'nın rastgele yönleri var ve buna göre ölçüm yapar.
    for i in range(target):
        pair = alice_bits[i]+alice_directions[i]
        if(pair=='00'):
            q_comm += '0'
        elif(pair=='10'):
            q_comm += '1'
        elif(pair=='01'):
            q_comm += '+'
        elif(pair=='11'):
            q_comm += '-'
        bob_bits += BB84(*pair, bob_directions[i])    
    q_comm += '>'
    
    print(next(alice))
    print("  ", end='')
    for i in range(8):
        print('-', end='')
        sleep(0.15)
    print('  ', end='')
    for i in q_comm:
        print(i, end='')
        sleep(0.05)
    print('  ', end='')
    for i in range(7):
        print('-', end='')
        sleep(0.15)
    print('>\n')
    
    c2 = '\t'*5+' '*4 # yani 44 boş karakter, bizi 2. sütuna taşıyor
    sleep(2)
    print(c2+"- Ayşe'nin mesajıyla aynı boyda\n"+c2+"  rastgele yön belirliyorum:")
    bqs = polarizer(bob_directions)
    i = 0
    while(i*32<len(bqs)):
        print(c2+'    '+bqs[32*i:min(32*(i+1),len(bqs))])
        i += 1
        
    sleep(2)
    print('\n'+c2+"- Bu rastgele yöne göre ölçümlerim:")
    i = 0
    while(i*32<len(bob_bits)):
        print(c2+'    '+bob_bits[32*i:min(32*(i+1),len(bob_bits))])
        i += 1
        
    sleep(2)
    print('\n\t\tRastgele yönlerimizi ortak kanalda karşılaştırıyoruz:')
    print('  -Ayşe:\t' + aqs)
    sleep(0.5)
    print('\t\t' + bqs + '\t:Bora-')
    print('\t\t', end='')
    for i in zip(alice_directions, bob_directions):
        if(i[0]==i[1]):
            print('=', end='')
            sleep(0.1)
        else:
            print(' ', end='')
            
    sleep(2)
    print('\n\t\tAynı yönlerde bit\'ler de aynı olacaktır:\n\t\t', end='')
    for i in zip(alice_directions, bob_directions):
        if(i[0]==i[1]):
            print('\u2193', end='')
            sleep(0.05)
        else:
            print(' ', end='')
    print('\n  -Ayşe:\t' + alice_bits)
    print('\t\t' + bob_bits + '\t:Bora-\n\t\t', end='')
    for i in zip(alice_directions, bob_directions):
        if(i[0]==i[1]):
            print('\u2193', end='')
            sleep(0.05)
        else:
            print(' ', end='')
    print('\n\t\t', end='')
    count = 0
    for i in zip(alice_directions, bob_directions, alice_bits, bob_bits):
        if(i[0]==i[1]):
            print(i[2], end='')
            sleep(0.15)
            count += 1
        else:
            print(' ', end='')
            
    print('\n\n- Bunun sonunda ikimizin de elinde,\n'+'  {} bit\'lik bir dizi oldu.'.format(count))
    print('  (Hedefimiz ise {} bit\'lik bir diziydi.)\n'.format(target_bits))
    sleep(1)
    print(c2+'- Eğer kusur veya dinlenme riski varsa,')
    print(c2+'  bunun belli bir kısmı karşılaştırılabilir.\n')
    sleep(1)
    print('- Uyuşmazlık belli bir oranın üzerindeyse,\n  bunu kullanmak yerine baştan başlarız.')

    # Demo'nun geliştirilmesi için,
    #   * kıyas adımı gösterilebilir
    #   * yeni protokoller eklenebilir